# Module 03: Album Art Management

**Difficulty**: ⭐⭐
**Estimated Time**: 45 minutes
**Prerequisites**: [Module 00](00_music_library_manager.ipynb), [Module 01](01_metadata_management.ipynb)

## Learning Objectives

By the end of this notebook, you will be able to:

1. Extract album art from audio files and save as image files
2. View and analyze album artwork (resolution, format, file size)
3. Find files with missing or low-quality album art
4. Embed album art into audio files
5. Perform batch operations for album art extraction and embedding
6. Organize and export album art collections

## Why Album Art Matters

Album art is more than just decoration:

- **Visual Organization**: Helps you quickly identify albums in your library
- **Media Player Display**: Shows up in music players, car stereos, and smart devices
- **Professional Appearance**: Makes your library look complete and well-maintained
- **Metadata Completeness**: Part of a properly tagged music collection

## What This Module Covers

1. **Extracting Album Art**
   - Extract embedded artwork from audio files
   - Save as JPG or PNG files
   - Batch extraction for entire library

2. **Analyzing Album Art**
   - View artwork in the notebook
   - Check resolution and quality
   - Identify missing artwork

3. **Embedding Album Art**
   - Add artwork to files that are missing it
   - Replace low-quality artwork
   - Batch embed from image files

4. **Album Art Library**
   - Export unique album art collection
   - Organize by artist or album
   - Create visual catalog

---

## Setup

Import required libraries and configure settings.

**Note**: This module requires the `Pillow` library for image processing. Install it if needed:
```bash
pip install Pillow
```

In [ ]:
# Standard library imports
import os
import sys
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Optional, Tuple
import hashlib
import io

# Third-party imports
import pandas as pd
from IPython.display import display, Image as IPythonImage, HTML
from mutagen import File
from mutagen.id3 import ID3, APIC
from mutagen.flac import FLAC, Picture
from mutagen.mp4 import MP4, MP4Cover

# Image processing
try:
    from PIL import Image
    PIL_AVAILABLE = True
except ImportError:
    PIL_AVAILABLE = False
    print("⚠️  PIL (Pillow) not installed. Some features will be limited.")
    print("   Install with: pip install Pillow")

In [ ]:
# Configuration
MUSIC_LIBRARY_PATH = Path('../../music').resolve()
ALBUM_ART_EXPORT_PATH = MUSIC_LIBRARY_PATH / 'album_art_collection'

# Supported audio file extensions
AUDIO_EXTENSIONS = {'.mp3', '.flac', '.wav', '.m4a', '.aac', '.ogg', '.wma', '.opus'}

# Print configuration
print(f"Music Library Path: {MUSIC_LIBRARY_PATH}")
print(f"Library exists: {MUSIC_LIBRARY_PATH.exists()}")
print(f"Album art export path: {ALBUM_ART_EXPORT_PATH}")
print(f"PIL/Pillow available: {PIL_AVAILABLE}")

---

## Part 1: Extracting Album Art

Learn how to extract embedded album artwork from audio files.

### How Album Art is Stored

Different audio formats store album art differently:

- **MP3**: ID3 tags (APIC frame)
- **FLAC**: Picture metadata block
- **M4A/AAC**: MP4 'covr' atom
- **OGG/Opus**: METADATA_BLOCK_PICTURE

Mutagen handles all these formats transparently.

In [ ]:
def extract_album_art(file_path: Path) -> Optional[Dict]:
    """
    Extract album art from an audio file.
    
    Returns:
        Dictionary with:
        - data: Image data (bytes)
        - mime_type: MIME type (e.g., 'image/jpeg')
        - description: Image description (if available)
        - type: Picture type (e.g., 'Cover (front)')
        
        Returns None if no album art found.
    """
    try:
        audio = File(file_path)
        
        if audio is None:
            return None
        
        artwork = None
        
        # MP3 files with ID3 tags
        if file_path.suffix.lower() == '.mp3':
            if hasattr(audio, 'tags') and audio.tags:
                # Look for APIC (Attached Picture) frames
                for tag in audio.tags.values():
                    if isinstance(tag, APIC):
                        artwork = {
                            'data': tag.data,
                            'mime_type': tag.mime,
                            'description': tag.desc,
                            'type': str(tag.type)
                        }
                        break
        
        # FLAC files
        elif file_path.suffix.lower() == '.flac':
            if hasattr(audio, 'pictures') and audio.pictures:
                pic = audio.pictures[0]
                artwork = {
                    'data': pic.data,
                    'mime_type': pic.mime,
                    'description': pic.desc,
                    'type': str(pic.type)
                }
        
        # M4A/AAC files
        elif file_path.suffix.lower() in {'.m4a', '.mp4'}:
            if hasattr(audio, 'tags') and audio.tags and 'covr' in audio.tags:
                cover = audio.tags['covr'][0]
                # Determine MIME type from imageformat
                mime_type = 'image/jpeg' if cover.imageformat == MP4Cover.FORMAT_JPEG else 'image/png'
                artwork = {
                    'data': bytes(cover),
                    'mime_type': mime_type,
                    'description': '',
                    'type': 'Cover'
                }
        
        # OGG files
        elif file_path.suffix.lower() in {'.ogg', '.opus'}:
            if hasattr(audio, 'tags') and audio.tags:
                # OGG uses METADATA_BLOCK_PICTURE in vorbis comments
                if 'metadata_block_picture' in audio.tags:
                    import base64
                    pic_data = base64.b64decode(audio.tags['metadata_block_picture'][0])
                    # Parse the FLAC picture block
                    pic = Picture(pic_data)
                    artwork = {
                        'data': pic.data,
                        'mime_type': pic.mime,
                        'description': pic.desc,
                        'type': str(pic.type)
                    }
        
        return artwork
    
    except Exception as e:
        # Silently fail for files without artwork or with errors
        return None

In [ ]:
def save_album_art(artwork_data: bytes, output_path: Path, mime_type: str = 'image/jpeg'):
    """
    Save album art data to an image file.
    
    Args:
        artwork_data: Binary image data
        output_path: Where to save the image
        mime_type: MIME type to determine file extension
    """
    # Determine extension from MIME type
    if 'jpeg' in mime_type or 'jpg' in mime_type:
        ext = '.jpg'
    elif 'png' in mime_type:
        ext = '.png'
    else:
        ext = '.jpg'  # Default to JPG
    
    # Ensure output path has correct extension
    if output_path.suffix.lower() not in {'.jpg', '.jpeg', '.png'}:
        output_path = output_path.with_suffix(ext)
    
    # Create parent directory if needed
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Save the image
    with open(output_path, 'wb') as f:
        f.write(artwork_data)

In [ ]:
def get_artwork_info(artwork_data: bytes) -> Dict:
    """
    Get information about album artwork (size, resolution, format).
    
    Requires PIL/Pillow.
    """
    if not PIL_AVAILABLE:
        return {
            'size_bytes': len(artwork_data),
            'size_kb': len(artwork_data) / 1024,
            'resolution': 'Unknown (PIL not available)',
            'format': 'Unknown'
        }
    
    try:
        img = Image.open(io.BytesIO(artwork_data))
        
        return {
            'size_bytes': len(artwork_data),
            'size_kb': len(artwork_data) / 1024,
            'width': img.width,
            'height': img.height,
            'resolution': f"{img.width}x{img.height}",
            'format': img.format,
            'mode': img.mode,  # RGB, RGBA, etc.
            'quality_tier': categorize_artwork_quality(img.width, img.height)
        }
    except Exception as e:
        return {
            'size_bytes': len(artwork_data),
            'size_kb': len(artwork_data) / 1024,
            'error': str(e)
        }


def categorize_artwork_quality(width: int, height: int) -> str:
    """
    Categorize artwork quality based on resolution.
    """
    min_dimension = min(width, height)
    
    if min_dimension >= 1000:
        return 'High (1000px+)'
    elif min_dimension >= 500:
        return 'Medium (500-999px)'
    elif min_dimension >= 300:
        return 'Low (300-499px)'
    else:
        return 'Very Low (<300px)'

### 🎯 Exercise 1: Extract and View Album Art

Let's extract album art from a file and view it.

In [ ]:
# Find a file with album art to test
test_file = None

for root, dirs, files in os.walk(MUSIC_LIBRARY_PATH):
    for file in files:
        file_path = Path(root) / file
        if file_path.suffix.lower() in AUDIO_EXTENSIONS:
            artwork = extract_album_art(file_path)
            if artwork:
                test_file = file_path
                break
    if test_file:
        break

if test_file:
    print(f"Found file with album art: {test_file.name}")
    artwork = extract_album_art(test_file)
    
    # Get artwork info
    info = get_artwork_info(artwork['data'])
    print(f"\nArtwork Info:")
    print(f"  Size: {info['size_kb']:.1f} KB")
    if 'resolution' in info:
        print(f"  Resolution: {info['resolution']}")
    if 'format' in info:
        print(f"  Format: {info['format']}")
    if 'quality_tier' in info:
        print(f"  Quality: {info['quality_tier']}")
    
    # Display the artwork
    print("\nAlbum Artwork:")
    display(IPythonImage(data=artwork['data']))
else:
    print("No files with album art found in library.")

---

## Part 2: Scanning Library for Album Art

Scan your entire library to see which files have album art and which don't.

In [ ]:
def scan_library_for_album_art(library_path: Path) -> List[Dict]:
    """
    Scan library and check album art status for all files.
    
    Returns list of dictionaries with file info and artwork status.
    """
    print(f"Scanning library: {library_path}")
    print("Checking album art status...\n")
    
    results = []
    file_count = 0
    
    for root, dirs, files in os.walk(library_path):
        for file in files:
            file_path = Path(root) / file
            
            if file_path.suffix.lower() in AUDIO_EXTENSIONS:
                # Extract basic metadata
                audio = File(file_path, easy=True)
                artist = 'Unknown'
                album = 'Unknown'
                
                if audio:
                    if 'artist' in audio:
                        artist = str(audio['artist'][0]) if isinstance(audio['artist'], list) else str(audio['artist'])
                    if 'album' in audio:
                        album = str(audio['album'][0]) if isinstance(audio['album'], list) else str(audio['album'])
                
                # Check for album art
                artwork = extract_album_art(file_path)
                
                file_info = {
                    'filename': file,
                    'path': str(file_path.relative_to(library_path)),
                    'artist': artist,
                    'album': album,
                    'has_artwork': artwork is not None,
                    'format': file_path.suffix.lower()
                }
                
                if artwork:
                    info = get_artwork_info(artwork['data'])
                    file_info['artwork_size_kb'] = info['size_kb']
                    file_info['artwork_resolution'] = info.get('resolution', 'Unknown')
                    file_info['artwork_quality'] = info.get('quality_tier', 'Unknown')
                else:
                    file_info['artwork_size_kb'] = 0
                    file_info['artwork_resolution'] = 'None'
                    file_info['artwork_quality'] = 'Missing'
                
                results.append(file_info)
                file_count += 1
                
                if file_count % 100 == 0:
                    print(f"Scanned {file_count} files...")
    
    print(f"\n✓ Scan complete: {file_count} files")
    return results

In [ ]:
def get_album_art_statistics(scan_results: List[Dict]) -> Dict:
    """
    Calculate statistics about album art in the library.
    """
    total_files = len(scan_results)
    files_with_art = sum(1 for f in scan_results if f['has_artwork'])
    files_without_art = total_files - files_with_art
    
    # Quality distribution
    quality_counts = {}
    for result in scan_results:
        quality = result['artwork_quality']
        quality_counts[quality] = quality_counts.get(quality, 0) + 1
    
    return {
        'total_files': total_files,
        'files_with_artwork': files_with_art,
        'files_without_artwork': files_without_art,
        'coverage_percentage': (files_with_art / total_files * 100) if total_files > 0 else 0,
        'quality_distribution': quality_counts
    }


def print_album_art_statistics(stats: Dict):
    """
    Print formatted album art statistics.
    """
    print("=" * 60)
    print("ALBUM ART STATISTICS")
    print("=" * 60)
    print(f"Total files:            {stats['total_files']:,}")
    print(f"Files with artwork:     {stats['files_with_artwork']:,} ({stats['coverage_percentage']:.1f}%)")
    print(f"Files without artwork:  {stats['files_without_artwork']:,}")
    print()
    print("Quality Distribution:")
    for quality, count in sorted(stats['quality_distribution'].items()):
        print(f"  {quality:20s}: {count:,}")
    print("=" * 60)

### 🎯 Exercise 2: Scan Library for Album Art

In [ ]:
# Scan the library
artwork_scan = scan_library_for_album_art(MUSIC_LIBRARY_PATH)

# Get statistics
stats = get_album_art_statistics(artwork_scan)
print_album_art_statistics(stats)

In [ ]:
# View files without artwork
missing_artwork = [f for f in artwork_scan if not f['has_artwork']]

if missing_artwork:
    print(f"\nFiles missing album art: {len(missing_artwork)}\n")
    df = pd.DataFrame(missing_artwork)
    display(df[['filename', 'artist', 'album', 'format']].head(20))
else:
    print("\n✓ All files have album art!")

In [ ]:
# View files with low-quality artwork
low_quality = [f for f in artwork_scan if f['has_artwork'] and 
               ('Very Low' in f['artwork_quality'] or 'Low' in f['artwork_quality'])]

if low_quality:
    print(f"\nFiles with low-quality artwork: {len(low_quality)}\n")
    df = pd.DataFrame(low_quality)
    display(df[['filename', 'artist', 'album', 'artwork_resolution', 'artwork_quality']].head(20))
else:
    print("\n✓ No low-quality artwork found!")

---

## Part 3: Batch Extract Album Art

Export all unique album artwork from your library.

In [ ]:
def batch_extract_album_art(library_path: Path, output_path: Path, 
                           organize_by: str = 'artist_album') -> List[Dict]:
    """
    Extract all album art from library and save to files.
    
    Args:
        library_path: Music library root
        output_path: Where to save extracted artwork
        organize_by: How to organize output
            - 'flat': All images in one folder (numbered)
            - 'artist': Organize by artist name
            - 'artist_album': Organize by artist/album (default)
    
    Returns:
        List of extraction results
    """
    print(f"Extracting album art from: {library_path}")
    print(f"Saving to: {output_path}")
    print(f"Organization: {organize_by}\n")
    
    output_path.mkdir(parents=True, exist_ok=True)
    
    results = []
    extracted_count = 0
    seen_artworks = set()  # Track unique artworks by hash
    
    for root, dirs, files in os.walk(library_path):
        for file in files:
            file_path = Path(root) / file
            
            if file_path.suffix.lower() in AUDIO_EXTENSIONS:
                artwork = extract_album_art(file_path)
                
                if artwork:
                    # Calculate hash to detect duplicates
                    art_hash = hashlib.md5(artwork['data']).hexdigest()
                    
                    # Skip if we've already extracted this exact artwork
                    if art_hash in seen_artworks:
                        continue
                    
                    seen_artworks.add(art_hash)
                    
                    # Get metadata for naming
                    audio = File(file_path, easy=True)
                    artist = 'Unknown Artist'
                    album = 'Unknown Album'
                    
                    if audio:
                        if 'artist' in audio:
                            artist = str(audio['artist'][0]) if isinstance(audio['artist'], list) else str(audio['artist'])
                        if 'album' in audio:
                            album = str(audio['album'][0]) if isinstance(audio['album'], list) else str(audio['album'])
                    
                    # Sanitize names for file system
                    artist_safe = sanitize_filename(artist)
                    album_safe = sanitize_filename(album)
                    
                    # Determine output path based on organization
                    if organize_by == 'flat':
                        out_file = output_path / f"artwork_{extracted_count:04d}.jpg"
                    elif organize_by == 'artist':
                        artist_folder = output_path / artist_safe
                        artist_folder.mkdir(exist_ok=True)
                        out_file = artist_folder / f"{album_safe}.jpg"
                    else:  # artist_album
                        album_folder = output_path / artist_safe / album_safe
                        album_folder.mkdir(parents=True, exist_ok=True)
                        out_file = album_folder / "cover.jpg"
                    
                    # Save the artwork
                    save_album_art(artwork['data'], out_file, artwork['mime_type'])
                    
                    results.append({
                        'source_file': str(file_path.relative_to(library_path)),
                        'artist': artist,
                        'album': album,
                        'output_file': str(out_file.relative_to(output_path)),
                        'artwork_hash': art_hash
                    })
                    
                    extracted_count += 1
                    
                    if extracted_count % 50 == 0:
                        print(f"Extracted {extracted_count} unique artworks...")
    
    print(f"\n✓ Extraction complete: {extracted_count} unique artworks saved")
    return results


def sanitize_filename(name: str) -> str:
    """
    Sanitize a string to be safe for use as a filename.
    """
    # Remove problematic characters
    name = name.replace('/', '_').replace('\\', '_')
    name = name.replace(':', '_').replace('*', '_')
    name = name.replace('?', '_').replace('"', "'")
    name = name.replace('<', '_').replace('>', '_')
    name = name.replace('|', '_')
    
    # Trim and limit length
    name = name.strip()
    if len(name) > 100:
        name = name[:100]
    
    return name if name else 'Unknown'

### 🎯 Exercise 3: Export Album Art Collection

In [ ]:
# Extract all album art organized by artist/album
extraction_results = batch_extract_album_art(
    MUSIC_LIBRARY_PATH,
    ALBUM_ART_EXPORT_PATH,
    organize_by='artist_album'
)

print(f"\nExtracted {len(extraction_results)} unique album artworks")
print(f"Saved to: {ALBUM_ART_EXPORT_PATH}")

In [ ]:
# View extraction results
if extraction_results:
    df = pd.DataFrame(extraction_results)
    display(df.head(20))

---

## Part 4: Embedding Album Art

Add album art to files that are missing it.

**Note**: Embedding album art modifies your audio files. Always test on copies first!

In [ ]:
def embed_album_art_mp3(file_path: Path, image_path: Path) -> bool:
    """
    Embed album art into an MP3 file.
    
    Returns True if successful.
    """
    try:
        audio = ID3(file_path)
        
        # Read image data
        with open(image_path, 'rb') as img_file:
            image_data = img_file.read()
        
        # Determine MIME type
        mime_type = 'image/jpeg' if image_path.suffix.lower() in {'.jpg', '.jpeg'} else 'image/png'
        
        # Create APIC frame
        audio['APIC'] = APIC(
            encoding=3,  # UTF-8
            mime=mime_type,
            type=3,  # Cover (front)
            desc='Cover',
            data=image_data
        )
        
        audio.save()
        return True
    
    except Exception as e:
        print(f"Error embedding artwork in {file_path.name}: {e}")
        return False


def embed_album_art_flac(file_path: Path, image_path: Path) -> bool:
    """
    Embed album art into a FLAC file.
    """
    try:
        audio = FLAC(file_path)
        
        # Read image
        if PIL_AVAILABLE:
            img = Image.open(image_path)
            img_byte_arr = io.BytesIO()
            img.save(img_byte_arr, format='JPEG' if image_path.suffix.lower() in {'.jpg', '.jpeg'} else 'PNG')
            image_data = img_byte_arr.getvalue()
        else:
            with open(image_path, 'rb') as img_file:
                image_data = img_file.read()
        
        # Create picture
        picture = Picture()
        picture.type = 3  # Cover (front)
        picture.mime = 'image/jpeg' if image_path.suffix.lower() in {'.jpg', '.jpeg'} else 'image/png'
        picture.desc = 'Cover'
        picture.data = image_data
        
        # Clear existing pictures and add new one
        audio.clear_pictures()
        audio.add_picture(picture)
        audio.save()
        return True
    
    except Exception as e:
        print(f"Error embedding artwork in {file_path.name}: {e}")
        return False


def embed_album_art(file_path: Path, image_path: Path) -> bool:
    """
    Embed album art into an audio file (auto-detect format).
    """
    if file_path.suffix.lower() == '.mp3':
        return embed_album_art_mp3(file_path, image_path)
    elif file_path.suffix.lower() == '.flac':
        return embed_album_art_flac(file_path, image_path)
    elif file_path.suffix.lower() in {'.m4a', '.mp4'}:
        try:
            audio = MP4(file_path)
            with open(image_path, 'rb') as img_file:
                image_data = img_file.read()
            
            fmt = MP4Cover.FORMAT_JPEG if image_path.suffix.lower() in {'.jpg', '.jpeg'} else MP4Cover.FORMAT_PNG
            audio['covr'] = [MP4Cover(image_data, imageformat=fmt)]
            audio.save()
            return True
        except Exception as e:
            print(f"Error embedding artwork in {file_path.name}: {e}")
            return False
    else:
        print(f"Embedding not supported for format: {file_path.suffix}")
        return False

### 🎯 Exercise 4: Embed Album Art

**Warning**: This modifies your audio files. Test on a copy first!

In [ ]:
# Example: Embed album art into a file
# Uncomment and modify paths to test

# test_audio_file = MUSIC_LIBRARY_PATH / "path/to/your/file.mp3"
# test_image_file = MUSIC_LIBRARY_PATH / "path/to/cover.jpg"

# if test_audio_file.exists() and test_image_file.exists():
#     success = embed_album_art(test_audio_file, test_image_file)
#     if success:
#         print("✓ Album art embedded successfully!")
#     else:
#         print("✗ Failed to embed album art")
# else:
#     print("Files not found. Update the paths above.")

print("Embedding example commented out for safety.")
print("Uncomment the code above and provide file paths to test.")

---

## Summary

In this module, you learned:

### Key Concepts

1. **Album Art Extraction**
   - How different formats store album art
   - Extracting artwork with mutagen
   - Saving artwork as image files

2. **Album Art Analysis**
   - Checking resolution and quality
   - Finding missing artwork
   - Identifying low-quality artwork

3. **Batch Operations**
   - Extracting all unique artworks
   - Organizing artwork collections
   - Avoiding duplicate exports

4. **Embedding Album Art**
   - Adding artwork to MP3, FLAC, M4A files
   - Understanding format-specific methods
   - Safe modification practices

### Best Practices

- **Target resolution**: Aim for 500x500px minimum, 1000x1000px for high quality
- **File size**: Keep album art under 500KB to avoid bloating audio files
- **Format**: JPEG is standard (smaller size), PNG for lossless quality
- **Backup first**: Always backup before embedding artwork
- **Consistency**: Use same artwork for all tracks in an album

### What's Next?

- **[Module 05: External API Integration](05_external_api_integration.ipynb)** - Auto-download album art from online sources
- **[Module 06: Library Visualizations](06_library_visualizations.ipynb)** - Create visual reports

### Additional Resources

- **Album Art Sources**: Musicbrainz, Last.fm, iTunes, Discogs
- **Image Tools**: Pillow documentation for advanced image processing
- **Mutagen Docs**: [https://mutagen.readthedocs.io/](https://mutagen.readthedocs.io/)

---

**Excellent work!** Your library can now have beautiful, complete album artwork. 🎨